In [1]:
import os, sys
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
dmethod = 1
#
if dmethod == 0:
    client = None
if dmethod == 1:
    from dask.distributed import Client
    scheduler = os.getenv('DATAWORK')+'/dask/scheduler.json'
    client = Client(scheduler_file=scheduler)
elif dmethod == 2:
    from dask_jobqueue import PBSCluster
    # folder where data is spilled when RAM is filled up
    local_dir = os.getenv('TMPDIR')
    #
    cluster = PBSCluster(queue='mpi_1', local_directory=local_dir, interface='ib0', walltime='24:00:00',
                         threads=14, processes=2, memory='50GB', resource_spec='select=1:ncpus=28:mem=100g', 
                         death_timeout=100)
    w = cluster.start_workers(40)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# you need to wait for workers to spin up
if dmethod == 2:
    cluster.scheduler

In [4]:
# get dask handles and check dask server status
if dmethod == 2:
    from dask.distributed import Client
    client = Client(cluster)

In [5]:
client

Client Scheduler: tcp://10.148.0.148:8786 Dashboard: http://10.148.0.148:8787/status,Cluster Workers: 7 Cores: 98 Memory: 700.00 GB


---

# store grid

In [6]:
ds_index, ds = get_compressed_level_index(grid_dir)

ds = ds.drop(['dxG','dyG','dxC','dyC','rAw','rAs','rAz'])
ds = ds.drop(['hFacC','hFacW','hFacS'])
ds = ds.drop(['Z', 'Zp1', 'Zu', 'Zl', 'drC', 'drF','PHrefC','PHrefF'])
print(ds)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:721: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)


<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * k_u      (k_u) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_l      (k_l) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_p1     (k_p1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC       (face, j, i) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>
    YC       (face, j, i) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 432

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:235: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


In [7]:
out_dir = scratch+'/'
print(out_dir)

file_out = out_dir+'mit_grid.nc'
ds.to_netcdf(file_out, mode='w')


/home1/scratch/xyu/


In [8]:
dst = ds.transpose('face', 'i', 'j', 'i_g', 'j_g', 'k', 'k_l', 'k_p1', 'k_u')
print(dst)

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * k_u      (k_u) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_l      (k_l) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * k_p1     (k_p1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC       (face, i, j) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>
    YC       (face, i, j) >f4 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 432

In [9]:
file_out = out_dir+'mit_grid_t.nc'
dst.to_netcdf(file_out, mode='w')

--- 

# automatic rewriting of all variables (for spectral calculations)

# transposed data: (i,j,time), 1 file per face, time windows

In [ ]:
V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['SSU']

transpose = True # False untested

if transpose:
    Nt = 24*10 # time windows to consider
    out_dir = scratch+'/mit_nc/'
    fsize_bound = 15*1e9
else:
    Nt = 1
    out_dir = scratch+'/mit_nc/'    
    fsize_bound = 60*1e6    

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    it = np.arange(time.size/Nt-1).astype(int)*Nt
    #it = np.arange(10).astype(int)*Nt # tmp
    assert it[-1]+Nt<time.size
    #
    p = 'C'
    if v is 'SSU':
        p = 'W'
    elif v is 'SSV':
        p = 'S'
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #ds = ds.chunk({'face': 1})
    #
    for face in range(ds['face'].size):
        for i, t in enumerate(it):
            #
            file_out = out_dir+'/%s_f%02d_t%02d.nc'%(v,face,i)
            if not os.path.isfile(file_out) or os.path.getsize(file_out) < fsize_bound:            
                dv = ds[v].isel(time=slice(t,t+Nt), face=face) 
                # should store grid data independantly in a single file
                dv = dv.drop(['XC','YC','Depth','rA']).to_dataset()
                #
                if transpose:
                    dv = dv.chunk({'time': dv['time'].size, 'i': 432, 'j': 432})
                    dv = dv.transpose('i','j','time')
                    chunksizes = [432, 432, dv['time'].size]
                else:
                    dv = dv.chunk({'i': 432, 'j': 432})
                    chunksizes = [1, 432, 432]
                #print(dv)
                #
                while True:
                    try:
                        %time dv.to_netcdf(file_out, mode='w', unlimited_dims=['time'], \
                                           encoding={'SSU': {'chunksizes': chunksizes}})
                    except:
                        print('Failure')
                    if os.path.isfile(file_out) and os.path.getsize(file_out)>fsize_bound:
                        #
                        print('face=%d / i=%d'%(face,i))
                        break
            else:
                print('face=%d / i=%d - allready processed'%(face,i))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:721: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:235: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/worker.py:742: UserWarning: Large object of size 3.14 MB detected in task graph: 
  ("('astype-2a905e7b46935b1a3b2a77d438b52a0b', 

CPU times: user 33.5 s, sys: 228 ms, total: 33.7 s
Wall time: 2min 29s
face=0 / i=0
CPU times: user 33.4 s, sys: 248 ms, total: 33.7 s
Wall time: 2min 22s
face=0 / i=1
CPU times: user 33 s, sys: 244 ms, total: 33.2 s
Wall time: 2min 21s
face=0 / i=2
CPU times: user 33.1 s, sys: 236 ms, total: 33.4 s
Wall time: 2min 23s
face=0 / i=3
CPU times: user 32.9 s, sys: 280 ms, total: 33.2 s
Wall time: 2min 21s
face=0 / i=4
CPU times: user 33 s, sys: 276 ms, total: 33.3 s
Wall time: 2min 22s
face=0 / i=5
CPU times: user 32.9 s, sys: 228 ms, total: 33.1 s
Wall time: 2min 23s
face=0 / i=6
CPU times: user 33.1 s, sys: 244 ms, total: 33.4 s
Wall time: 2min 26s
face=0 / i=7
CPU times: user 33 s, sys: 224 ms, total: 33.2 s
Wall time: 2min 22s
face=0 / i=8
CPU times: user 33.1 s, sys: 200 ms, total: 33.3 s
Wall time: 2min 23s
face=0 / i=9


--- 

# standard data layout: (face, j, i), 1 file per time (for movies)

In [ ]:
V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['SSU']

Nt = 1
#out_dir = datawork+'/mit_nc/'
out_dir = scratch+'/mit_nc_SSU/'    
fsize_bound = 13*60*1e6

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    it = np.arange(time.size/Nt-1).astype(int)*Nt
    #it = np.arange(10).astype(int)*Nt # tmp
    assert it[-1]+Nt<time.size
    #
    p = 'C'
    if v is 'SSU':
        p = 'W'
    elif v is 'SSV':
        p = 'S'
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #ds = ds.chunk({'face': 1})
    #
    for i, t in enumerate(it):
        #
        file_out = out_dir+'/%s_t%04d.nc'%(v,i)
        if not os.path.isfile(file_out) or os.path.getsize(file_out) < fsize_bound:            
            dv = ds[v].isel(time=slice(t,t+Nt)) 
            # should store grid data independantly in a single file
            dv = dv.drop(['XC','YC','Depth','rA']).to_dataset()
            #
            #print(dv)
            #
            while True:
                try:
                    #print(dv)
                    %time dv.to_netcdf(file_out, mode='w')                    
                except:
                    print('Failure')
                if os.path.isfile(file_out) and os.path.getsize(file_out) > fsize_bound:
                    #
                    print('i=%d, iter=%d'%(i, iters[i].values))
                    break
        else:
            print('i=%d, iter=%d - allready processed'%(i, iters[i].values))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:721: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:314: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:235: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:


i=0, iter=228096 - allready processed
i=1, iter=228240 - allready processed
i=2, iter=228384 - allready processed
i=3, iter=228528 - allready processed
i=4, iter=228672 - allready processed
i=5, iter=228816 - allready processed
i=6, iter=228960 - allready processed
i=7, iter=229104 - allready processed
i=8, iter=229248 - allready processed
i=9, iter=229392 - allready processed
i=10, iter=229536 - allready processed
i=11, iter=229680 - allready processed
i=12, iter=229824 - allready processed
i=13, iter=229968 - allready processed
i=14, iter=230112 - allready processed
i=15, iter=230256 - allready processed
i=16, iter=230400 - allready processed
i=17, iter=230544 - allready processed
i=18, iter=230688 - allready processed
i=19, iter=230832 - allready processed
i=20, iter=230976 - allready processed
i=21, iter=231120 - allready processed
i=22, iter=231264 - allready processed
i=23, iter=231408 - allready processed
i=24, iter=231552 - allready processed
i=25, iter=231696 - allready proces

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/worker.py:742: UserWarning: Large object of size 2.93 MB detected in task graph: 
  ("('astype-874b2b53ef110f390cfbed6c25b2a503', 0, 0 ... 88d0>]), False)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s))


CPU times: user 472 ms, sys: 12 ms, total: 484 ms
Wall time: 9.48 s
i=38, iter=233568
CPU times: user 380 ms, sys: 16 ms, total: 396 ms
Wall time: 5.07 s
i=39, iter=233712
CPU times: user 376 ms, sys: 24 ms, total: 400 ms
Wall time: 4.85 s
i=40, iter=233856
CPU times: user 380 ms, sys: 20 ms, total: 400 ms
Wall time: 4.69 s
i=41, iter=234000
CPU times: user 392 ms, sys: 20 ms, total: 412 ms
Wall time: 4.68 s
i=42, iter=234144
CPU times: user 368 ms, sys: 24 ms, total: 392 ms
Wall time: 5.88 s
i=43, iter=234288
CPU times: user 368 ms, sys: 20 ms, total: 388 ms
Wall time: 5.76 s
i=44, iter=234432
CPU times: user 380 ms, sys: 20 ms, total: 400 ms
Wall time: 4.93 s
i=45, iter=234576
CPU times: user 376 ms, sys: 24 ms, total: 400 ms
Wall time: 5 s
i=46, iter=234720
CPU times: user 384 ms, sys: 20 ms, total: 404 ms
Wall time: 4.95 s
i=47, iter=234864
CPU times: user 388 ms, sys: 8 ms, total: 396 ms
Wall time: 4.79 s
i=48, iter=235008
CPU times: user 384 ms, sys: 24 ms, total: 408 ms
Wall tim

CPU times: user 448 ms, sys: 24 ms, total: 472 ms
Wall time: 4.75 s
i=134, iter=247392
CPU times: user 380 ms, sys: 16 ms, total: 396 ms
Wall time: 4.86 s
i=135, iter=247536
CPU times: user 376 ms, sys: 20 ms, total: 396 ms
Wall time: 4.52 s
i=136, iter=247680
CPU times: user 388 ms, sys: 16 ms, total: 404 ms
Wall time: 4.9 s
i=137, iter=247824
CPU times: user 376 ms, sys: 20 ms, total: 396 ms
Wall time: 4.86 s
i=138, iter=247968
CPU times: user 384 ms, sys: 16 ms, total: 400 ms
Wall time: 5 s
i=139, iter=248112
CPU times: user 380 ms, sys: 12 ms, total: 392 ms
Wall time: 4.75 s
i=140, iter=248256
CPU times: user 388 ms, sys: 16 ms, total: 404 ms
Wall time: 4.78 s
i=141, iter=248400
CPU times: user 376 ms, sys: 20 ms, total: 396 ms
Wall time: 4.83 s
i=142, iter=248544
CPU times: user 380 ms, sys: 20 ms, total: 400 ms
Wall time: 5.08 s
i=143, iter=248688
CPU times: user 368 ms, sys: 28 ms, total: 396 ms
Wall time: 4.92 s
i=144, iter=248832
CPU times: user 376 ms, sys: 16 ms, total: 392 

...

CPU times: user 384 ms, sys: 24 ms, total: 408 ms
Wall time: 4.7 s
i=609, iter=315792
CPU times: user 384 ms, sys: 16 ms, total: 400 ms
Wall time: 4.79 s
i=610, iter=315936
CPU times: user 376 ms, sys: 24 ms, total: 400 ms
Wall time: 5.24 s
i=611, iter=316080
CPU times: user 372 ms, sys: 20 ms, total: 392 ms
Wall time: 4.72 s
i=612, iter=316224
CPU times: user 380 ms, sys: 16 ms, total: 396 ms
Wall time: 4.83 s
i=613, iter=316368
CPU times: user 376 ms, sys: 24 ms, total: 400 ms
Wall time: 4.86 s
i=614, iter=316512
CPU times: user 384 ms, sys: 16 ms, total: 400 ms
Wall time: 4.98 s
i=615, iter=316656
CPU times: user 380 ms, sys: 24 ms, total: 404 ms
Wall time: 5.56 s
i=616, iter=316800
CPU times: user 388 ms, sys: 24 ms, total: 412 ms
Wall time: 4.69 s
i=617, iter=316944
CPU times: user 376 ms, sys: 24 ms, total: 400 ms
Wall time: 4.56 s
i=618, iter=317088
CPU times: user 376 ms, sys: 24 ms, total: 400 ms
Wall time: 4.75 s
i=619, iter=317232
CPU times: user 380 ms, sys: 20 ms, total: 4

CPU times: user 372 ms, sys: 24 ms, total: 396 ms
Wall time: 4.86 s
i=704, iter=329472
CPU times: user 372 ms, sys: 24 ms, total: 396 ms
Wall time: 5.39 s
i=705, iter=329616
CPU times: user 384 ms, sys: 20 ms, total: 404 ms
Wall time: 4.92 s
i=706, iter=329760
CPU times: user 360 ms, sys: 24 ms, total: 384 ms
Wall time: 10.8 s
i=707, iter=329904
CPU times: user 372 ms, sys: 32 ms, total: 404 ms
Wall time: 4.8 s
i=708, iter=330048
CPU times: user 392 ms, sys: 8 ms, total: 400 ms
Wall time: 5 s
i=709, iter=330192
CPU times: user 380 ms, sys: 24 ms, total: 404 ms
Wall time: 4.9 s
i=710, iter=330336
CPU times: user 384 ms, sys: 16 ms, total: 400 ms
Wall time: 5.01 s
i=711, iter=330480
CPU times: user 372 ms, sys: 24 ms, total: 396 ms
Wall time: 4.8 s
i=712, iter=330624
CPU times: user 380 ms, sys: 20 ms, total: 400 ms
Wall time: 5.33 s
i=713, iter=330768
CPU times: user 376 ms, sys: 28 ms, total: 404 ms
Wall time: 4.87 s
i=714, iter=330912
CPU times: user 384 ms, sys: 20 ms, total: 404 ms


CPU times: user 368 ms, sys: 32 ms, total: 400 ms
Wall time: 5.4 s
i=799, iter=343152
CPU times: user 372 ms, sys: 28 ms, total: 400 ms
Wall time: 5.01 s
i=800, iter=343296
CPU times: user 372 ms, sys: 28 ms, total: 400 ms
Wall time: 4.74 s
i=801, iter=343440
CPU times: user 368 ms, sys: 32 ms, total: 400 ms
Wall time: 4.65 s
i=802, iter=343584
CPU times: user 384 ms, sys: 32 ms, total: 416 ms
Wall time: 5.17 s
i=803, iter=343728
CPU times: user 388 ms, sys: 12 ms, total: 400 ms
Wall time: 5.06 s
i=804, iter=343872
CPU times: user 384 ms, sys: 20 ms, total: 404 ms
Wall time: 5.35 s
i=805, iter=344016
CPU times: user 384 ms, sys: 12 ms, total: 396 ms
Wall time: 4.75 s
i=806, iter=344160
CPU times: user 368 ms, sys: 32 ms, total: 400 ms
Wall time: 4.72 s
i=807, iter=344304
CPU times: user 372 ms, sys: 24 ms, total: 396 ms
Wall time: 4.96 s
i=808, iter=344448
CPU times: user 380 ms, sys: 16 ms, total: 396 ms
Wall time: 5.01 s
i=809, iter=344592
CPU times: user 380 ms, sys: 24 ms, total: 4

CPU times: user 376 ms, sys: 24 ms, total: 400 ms
Wall time: 4.92 s
i=894, iter=356832
CPU times: user 384 ms, sys: 16 ms, total: 400 ms
Wall time: 4.97 s
i=895, iter=356976
CPU times: user 388 ms, sys: 8 ms, total: 396 ms
Wall time: 4.74 s
i=896, iter=357120
CPU times: user 384 ms, sys: 16 ms, total: 400 ms
Wall time: 4.68 s
i=897, iter=357264
CPU times: user 368 ms, sys: 32 ms, total: 400 ms
Wall time: 4.86 s
i=898, iter=357408
CPU times: user 396 ms, sys: 4 ms, total: 400 ms
Wall time: 5.07 s
i=899, iter=357552
CPU times: user 380 ms, sys: 20 ms, total: 400 ms
Wall time: 5.3 s
i=900, iter=357696
CPU times: user 376 ms, sys: 16 ms, total: 392 ms
Wall time: 4.88 s
i=901, iter=357840
CPU times: user 384 ms, sys: 16 ms, total: 400 ms
Wall time: 4.95 s
i=902, iter=357984
CPU times: user 388 ms, sys: 12 ms, total: 400 ms
Wall time: 5.15 s
i=903, iter=358128
CPU times: user 388 ms, sys: 8 ms, total: 396 ms
Wall time: 4.88 s
i=904, iter=358272
CPU times: user 360 ms, sys: 36 ms, total: 396 

CPU times: user 388 ms, sys: 16 ms, total: 404 ms
Wall time: 4.93 s
i=989, iter=370512
CPU times: user 396 ms, sys: 8 ms, total: 404 ms
Wall time: 5.34 s
i=990, iter=370656
CPU times: user 384 ms, sys: 16 ms, total: 400 ms
Wall time: 4.8 s
i=991, iter=370800
CPU times: user 364 ms, sys: 28 ms, total: 392 ms
Wall time: 4.74 s
i=992, iter=370944
CPU times: user 384 ms, sys: 16 ms, total: 400 ms
Wall time: 4.94 s
i=993, iter=371088
CPU times: user 372 ms, sys: 24 ms, total: 396 ms
Wall time: 5.06 s
i=994, iter=371232
CPU times: user 372 ms, sys: 24 ms, total: 396 ms
Wall time: 4.67 s
i=995, iter=371376
CPU times: user 384 ms, sys: 20 ms, total: 404 ms
Wall time: 5.02 s
i=996, iter=371520
CPU times: user 372 ms, sys: 28 ms, total: 400 ms
Wall time: 4.95 s
i=997, iter=371664
CPU times: user 388 ms, sys: 12 ms, total: 400 ms
Wall time: 5.06 s
i=998, iter=371808
CPU times: user 388 ms, sys: 28 ms, total: 416 ms
Wall time: 5.57 s
i=999, iter=371952
CPU times: user 364 ms, sys: 32 ms, total: 39

In [1]:
client

NameError: name 'client' is not defined

In [ ]:
client.restart()

---

```
aponte/mit_tmp% ncdump -sh Eta_f01_4.nc

netcdf Eta_f01_4 {
dimensions:
	time = UNLIMITED ; // (24 currently)
	i = 4320 ;
	j = 4320 ;
variables:
	int64 i(i) ;
		i:standard_name = "x_grid_index" ;
		i:axis = "X" ;
		i:long_name = "x-dimension of the t grid" ;
		i:swap_dim = "XC" ;
		i:_Storage = "contiguous" ;
		i:_Endianness = "little" ;
	int64 j(j) ;
		j:standard_name = "y_grid_index" ;
		j:axis = "Y" ;
		j:long_name = "y-dimension of the t grid" ;
		j:swap_dim = "YC" ;
		j:_Storage = "contiguous" ;
		j:_Endianness = "little" ;
	int64 face ;
		face:standard_name = "face_index" ;
		face:_Endianness = "little" ;
	double time(time) ;
		time:_FillValue = NaN ;
		time:_Storage = "chunked" ;
		time:_ChunkSizes = 512 ;
	float Eta(i, j, time) ;
		Eta:_FillValue = NaNf ;
		Eta:coordinates = "face" ;
		Eta:_Storage = "chunked" ;
		Eta:_ChunkSizes = 432, 432, 24 ;

// global attributes:
		:_NCProperties = "version=1|netcdflibversion=4.6.1|hdf5libversion=1.10.1" ;
		:_Format = "netCDF-4" ;
}
```